In [1]:
'''
Online retail sales data from 01/12/2010 and 09/12/2011 is analysed to find assication between items
For this, association mining is applied to detect association between items. 
'''

#load library
import pandas as pd

In [2]:
#load the dataset
#Due to the error "utf-8 codec can't decode byte 0xa3 in position 28: invalid start byte"
#encoding = 'unicode_escape' is applied
df = pd.read_csv('datasets/online retail.csv',header=0,encoding = 'unicode_escape')

C:\Users\kmw41\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532619 entries, 0 to 532618
Data columns (total 8 columns):
InvoiceNo      532619 non-null object
StockCode      532619 non-null object
Description    531165 non-null object
Quantity       532619 non-null int64
InvoiceDate    532619 non-null object
UnitPrice      532619 non-null float64
CustomerID     397924 non-null float64
Country        532619 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 32.5+ MB


In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
'''
Data preprocessing 
### do in need to remove space?###
'''
#remove negative values in quantiy column as it is not necessaory in terms of analysis of association.
df.drop(df[df.Quantity < 0].index, inplace=True)
    
# remove inappropriate values in stockcode column
df.drop(df[df.StockCode == 'DCGS0073'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0069'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0068'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0067'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'DCGS0003'].index, inplace=True) # written as ebay
df.drop(df[df.StockCode == 'B'].index, inplace=True)
df.drop(df[df.StockCode == 'AMAZONEFEE'].index, inplace=True)
df.drop(df[df.StockCode == 'BANK CHARGES'].index, inplace=True)
df.drop(df[df.StockCode == 'C2'].index, inplace=True)
df.drop(df[df.StockCode == 'DOT'].index, inplace=True)
df.drop(df[df.StockCode == 'M'].index, inplace=True)
df.drop(df[df.StockCode == 'PADS'].index, inplace=True)
df.drop(df[df.StockCode == 'POST'].index, inplace=True)
df.drop(df[df.StockCode == 'S'].index, inplace=True)

#gift is a gift voucher so that it may be better not to remove it
#df.drop(df[df.StockCode == 'gift_0001_10'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_20'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_30'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_40'].index, inplace=True)
#df.drop(df[df.StockCode == 'gift_0001_50'].index, inplace=True)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528950 entries, 0 to 532618
Data columns (total 8 columns):
InvoiceNo      528950 non-null object
StockCode      528950 non-null object
Description    528364 non-null object
Quantity       528950 non-null int64
InvoiceDate    528950 non-null object
UnitPrice      528950 non-null float64
CustomerID     396370 non-null float64
Country        528950 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 36.3+ MB


In [7]:
#drop nan value in description
#MissingData = df.isnull().sum()
#print(MissingData.sort_values(ascending = False))
df = df.dropna(subset = ['Description'])

In [130]:
#print(df['Description'].unique())
#print(df['Description'].value_counts())

In [131]:
# grouping dataset based on InoviceNo and Description
transactions = df.groupby(['InvoiceNo'])['Description'].apply(list)
print(transactions.head(5))

InvoiceNo
536365    [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
536366    [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
536367    [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
536368    [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
536369                           [BATH BUILDING BLOCK WORD]
Name: Description, dtype: object


In [132]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

In [133]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.009, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2))
#results = list(apriori(transaction_list, min_support=0.009, min_confidence = 0.2, min_lift = 3, min_length = 1, max_length = 5))


# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'JAM MAKING SET PRINTED', '6 RIBBONS RUSTIC CHARM'}), support=0.01182483214750977, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUSTIC CHARM'}), items_add=frozenset({'JAM MAKING SET PRINTED'}), confidence=0.24712041884816752, lift=4.2444314280307465), OrderedStatistic(items_base=frozenset({'JAM MAKING SET PRINTED'}), items_add=frozenset({'6 RIBBONS RUSTIC CHARM'}), confidence=0.20309810671256454, lift=4.2444314280307465)]), RelationRecord(items=frozenset({'6 RIBBONS RUSTIC CHARM', 'JAM MAKING SET WITH JARS'}), support=0.010371780739553062, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUSTIC CHARM'}), items_add=frozenset({'JAM MAKING SET WITH JARS'}), confidence=0.21675392670157068, lift=3.8181596373432902)]), RelationRecord(items=frozenset({'6 RIBBONS RUSTIC CHARM', 'NATURAL SLATE HEART CHALKBOARD '}), support=0.009920833750876842, ordered_statistics=[OrderedStatistic(items_base=frozenset({'6 RIBBONS RUS

In [134]:
result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.head(20))

                             Left_side                          Right_side  \
0               6 RIBBONS RUSTIC CHARM              JAM MAKING SET PRINTED   
1               JAM MAKING SET PRINTED              6 RIBBONS RUSTIC CHARM   
2               6 RIBBONS RUSTIC CHARM            JAM MAKING SET WITH JARS   
3               6 RIBBONS RUSTIC CHARM     NATURAL SLATE HEART CHALKBOARD    
4               6 RIBBONS RUSTIC CHARM     PACK OF 72 RETROSPOT CAKE CASES   
5               6 RIBBONS RUSTIC CHARM     RECIPE BOX PANTRY YELLOW DESIGN   
6               6 RIBBONS RUSTIC CHARM           SCANDINAVIAN REDS RIBBONS   
7            SCANDINAVIAN REDS RIBBONS              6 RIBBONS RUSTIC CHARM   
8               6 RIBBONS RUSTIC CHARM   SET OF 3 CAKE TINS PANTRY DESIGN    
9      60 CAKE CASES DOLLY GIRL DESIGN         60 TEATIME FAIRY CAKE CASES   
10         60 TEATIME FAIRY CAKE CASES     60 CAKE CASES DOLLY GIRL DESIGN   
11     60 CAKE CASES DOLLY GIRL DESIGN  PACK OF 60 PINK PAISLEY 

In [135]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(15))

                Left_side            Right_side   Support  Confidence  \
424     HERB MARKER THYME  HERB MARKER ROSEMARY  0.011073    0.932489   
423  HERB MARKER ROSEMARY     HERB MARKER THYME  0.011073    0.920833   
407   HERB MARKER CHIVES    HERB MARKER PARSLEY  0.009670    0.923445   
408   HERB MARKER PARSLEY   HERB MARKER CHIVES   0.009670    0.810924   
412     HERB MARKER THYME   HERB MARKER CHIVES   0.009470    0.797468   
411   HERB MARKER CHIVES      HERB MARKER THYME  0.009470    0.904306   
405   HERB MARKER CHIVES       HERB MARKER MINT  0.009520    0.909091   
406      HERB MARKER MINT   HERB MARKER CHIVES   0.009520    0.791667   
422     HERB MARKER THYME   HERB MARKER PARSLEY  0.010672    0.898734   
421   HERB MARKER PARSLEY     HERB MARKER THYME  0.010672    0.894958   
409   HERB MARKER CHIVES   HERB MARKER ROSEMARY  0.009470    0.904306   
410  HERB MARKER ROSEMARY   HERB MARKER CHIVES   0.009470    0.787500   
419   HERB MARKER PARSLEY  HERB MARKER ROSEMARY  0.

In [136]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Confidence', ascending=False)
print(result_df.head(15))

                   Left_side                Right_side   Support  Confidence  \
424        HERB MARKER THYME      HERB MARKER ROSEMARY  0.011073    0.932489   
407      HERB MARKER CHIVES        HERB MARKER PARSLEY  0.009670    0.923445   
423     HERB MARKER ROSEMARY         HERB MARKER THYME  0.011073    0.920833   
1315  REGENCY TEA PLATE PINK  REGENCY TEA PLATE GREEN   0.013929    0.911475   
405      HERB MARKER CHIVES           HERB MARKER MINT  0.009520    0.909091   
411      HERB MARKER CHIVES          HERB MARKER THYME  0.009470    0.904306   
409      HERB MARKER CHIVES       HERB MARKER ROSEMARY  0.009470    0.904306   
419      HERB MARKER PARSLEY      HERB MARKER ROSEMARY  0.010773    0.903361   
422        HERB MARKER THYME       HERB MARKER PARSLEY  0.010672    0.898734   
420     HERB MARKER ROSEMARY       HERB MARKER PARSLEY  0.010773    0.895833   
421      HERB MARKER PARSLEY         HERB MARKER THYME  0.010672    0.894958   
402     HERB MARKER ROSEMARY         HER

In [137]:
search = result_df[result_df['Left_side'] == 'HERB MARKER CHIVES ']
print(search)

               Left_side            Right_side   Support  Confidence  \
407  HERB MARKER CHIVES    HERB MARKER PARSLEY  0.009670    0.923445   
405  HERB MARKER CHIVES       HERB MARKER MINT  0.009520    0.909091   
411  HERB MARKER CHIVES      HERB MARKER THYME  0.009470    0.904306   
409  HERB MARKER CHIVES   HERB MARKER ROSEMARY  0.009470    0.904306   
396  HERB MARKER CHIVES      HERB MARKER BASIL  0.009119    0.870813   

          Lift  
407  77.437457  
405  75.598485  
411  76.152504  
409  75.200598  
396  71.816916  


# Sequence Analysis 

In [9]:
# convert time/date into a sepcific format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format = '%d/%m/%Y %H:%M')

In [10]:
#transactions = df.groupby(['CustomerID','InvoiceDate'])['Description'].apply(list)
#transactions.head(10)
#StockCode

In [11]:
transactions = df.groupby(['CustomerID','InvoiceDate'])['Description'].apply(list)
sequences = transactions.values.tolist()
#StockCode
#Description
# show the first 5 sequences
print(sequences[:10])

[['MEDIUM CERAMIC TOP STORAGE JAR'], ['BLACK CANDELABRA T-LIGHT HOLDER', 'AIRLINE BAG VINTAGE JET SET BROWN', 'COLOUR GLASS. STAR T-LIGHT HOLDER', 'MINI PAINT SET VINTAGE ', 'CLEAR DRAWER KNOB ACRYLIC EDWARDIAN', 'PINK DRAWER KNOB ACRYLIC EDWARDIAN', 'GREEN DRAWER KNOB ACRYLIC EDWARDIAN', 'RED DRAWER KNOB ACRYLIC EDWARDIAN', 'PURPLE DRAWERKNOB ACRYLIC EDWARDIAN', 'BLUE DRAWER KNOB ACRYLIC EDWARDIAN', 'ALARM CLOCK BAKELIKE CHOCOLATE', 'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE ORANGE', 'FOUR HOOK  WHITE LOVEBIRDS', 'BLACK GRAND BAROQUE PHOTO FRAME', 'BATHROOM METAL SIGN ', 'LARGE HEART MEASURING SPOONS', 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'BLUE 3 PIECE POLKADOT CUTLERY SET', 'RED 3 PIECE RETROSPOT CUTLERY SET', 'PINK 3 PIECE POLKADOT CUTLERY SET', 'EMERGENCY FIRST AID TIN ', 'SET OF 2 TINS VINTAGE BATHROOM ', 'SET/3 DECOUPAGE STACKING TINS', 'BOOM BOX SPEAKER BOYS', 'RED TOADSTOOL LED NIGHT LIGHT', '3D DOG PICTURE PLA

In [13]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    #supp_param = '{}%'.format(int(min_sup * 100))
    #conf_param = '{}%'.format(int(min_conf * 100))
    #subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', '10%', '10%'], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [14]:
result = get_association_rules(sequences, 0.009, 0.2)

In [15]:
# sort all acquired rules descending by support
result = result.sort_values(by='Support', ascending=False)
print(result.head(10))

                                        Left_rule  \
19           [BOX OF 6 ASSORTED COLOUR TEASPOONS]   
68                          [TOOTHPASTE TUBE PEN]   
55                       [WOODLAND CHARLOTTE BAG]   
59                       [WOODLAND CHARLOTTE BAG]   
57                       [WOODLAND CHARLOTTE BAG]   
62  [WOODLAND CHARLOTTE BAG, TOOTHPASTE TUBE PEN]   
64                         [CHOCOLATE CALCULATOR]   
66                         [CHOCOLATE CALCULATOR]   
67    [CHOCOLATE CALCULATOR, TOOTHPASTE TUBE PEN]   
65                         [CHOCOLATE CALCULATOR]   

                                           Right_rule   Support  Confidence  
19                           [WOODLAND CHARLOTTE BAG]  0.011904    0.861660  
68                  [AIRLINE BAG VINTAGE JET SET RED]  0.009228    0.949438  
55                              [TOOTHPASTE TUBE PEN]  0.008955    0.589928  
59                  [AIRLINE BAG VINTAGE JET SET RED]  0.008792    0.579137  
57  [TOOTHPASTE TUBE PEN, 

In [ ]:
export_csv = result.to_csv (r'C:\Users\kmw41\Desktop\0_case2\result_support.csv', index = None, header=True) 

In [16]:
# sort all acquired rules descending by confidence
result = result.sort_values(by='Confidence', ascending=False)
print(result.head(20))

                                            Left_rule  \
45  [BOX OF 6 ASSORTED COLOUR TEASPOONS, CHOCOLATE...   
46  [BOX OF 6 ASSORTED COLOUR TEASPOONS, TOOTHPAST...   
38  [BOX OF 6 ASSORTED COLOUR TEASPOONS, WOODLAND ...   
50  [BOX OF 6 ASSORTED COLOUR TEASPOONS, WOODLAND ...   
49  [BOX OF 6 ASSORTED COLOUR TEASPOONS, WOODLAND ...   
42  [BOX OF 6 ASSORTED COLOUR TEASPOONS, WOODLAND ...   
44  [BOX OF 6 ASSORTED COLOUR TEASPOONS, CHOCOLATE...   
48  [BOX OF 6 ASSORTED COLOUR TEASPOONS, WOODLAND ...   
40  [BOX OF 6 ASSORTED COLOUR TEASPOONS, CHOCOLATE...   
36  [BOX OF 6 ASSORTED COLOUR TEASPOONS, CHOCOLATE...   
56     [WOODLAND CHARLOTTE BAG, CHOCOLATE CALCULATOR]   
60     [WOODLAND CHARLOTTE BAG, CHOCOLATE CALCULATOR]   
61  [WOODLAND CHARLOTTE BAG, CHOCOLATE CALCULATOR,...   
58     [WOODLAND CHARLOTTE BAG, CHOCOLATE CALCULATOR]   
62      [WOODLAND CHARLOTTE BAG, TOOTHPASTE TUBE PEN]   
67        [CHOCOLATE CALCULATOR, TOOTHPASTE TUBE PEN]   
18                   [PACK OF 6

In [18]:
export_csv = result.to_csv (r'C:\Users\kmw41\Desktop\0_case2\result_confidence.csv', index = None, header=True) 